# Download, plot near-real-time DUACS SSH product

Tom Farrar, started 10/9/2022

* Download with motuclient
* Plot latest map
* make movie of longer time
* extract U, V time series at some point

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import functions  # requires functions.py from this directory
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
# import cftime


In [2]:
# %matplotlib inline  
%matplotlib qt5  
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' + 'SSH_plots/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = False
if zoom:
    xmin, xmax = (-126,-121)
    ymin, ymax = (36.25,38.5)
    levels = np.linspace(-.2,.2,41)
else:
    xmin, xmax = (-131,-121)
    ymin, ymax = (34, 43)
    levels = np.linspace(-.3,.3,41)

In [4]:
# A shell script using the motuclient, https://help.marine.copernicus.eu/en/articles/4796533-what-are-the-motu-client-motuclient-and-python-requirements
# Shell script from Ben Greenwood (email 9/21/2022)
'''
echo "$(date -u) download_aviso.sh" >> ./aviso_download.log

start="2022-09-15 00:00:00"
end="$(date -u "+%Y-%m-%d %H:%M:%S")"
out_dir='./'

# download data
motuclient --motu https://nrt.cmems-du.eu/motu-web/Motu --service-id SEALEVEL_GLO_PHY_L4_NRT_OBSERVATIONS_008_046-TDS --product-id dataset-duacs-nrt-global-merged-allsat-phy-l4 --longitude-min -140 --longitude-max -120 --latitude-min 34 --latitude-max 43 --date-min "$start" --date-max "$end" --variable adt --variable err_ugosa --variable err_vgosa --variable sla --variable ugos --variable ugosa --variable vgos --variable vgosa --out-dir "$out_dir" --out-name aviso.nc --user ***** --pwd *****
'''
! bash ~/download_aviso.sh


2022-10-12 16:06:45.223 [ INFO] Asynchronous mode set
2022-10-12 16:06:45.223 [ INFO] Authenticating user tfarrar1 for service https://nrt.cmems-du.eu/motu-web/Motu
2022-10-12 16:06:47.394 [ INFO] Requesting file to download (this can take a while)...
2022-10-12 16:06:53.459 [ INFO] Authenticating user tfarrar1 for service https://nrt.cmems-du.eu/motu-web/Motu
2022-10-12 16:07:01.336 [ INFO] Product is not yet available (request in progress)
2022-10-12 16:07:11.347 [ INFO] Authenticating user tfarrar1 for service https://nrt.cmems-du.eu/motu-web/Motu
2022-10-12 16:07:19.230 [ INFO] The product is ready for download
2022-10-12 16:07:19.231 [ INFO] Downloading file (this can take a while)...
2022-10-12 16:07:19.676 [ INFO] File type: application/x-netcdf
2022-10-12 16:07:19.676 [ INFO] File size: 81.3 MB (81337208 B)
2022-10-12 16:07:19.676 [ INFO] Downloading file /home/jtomf/Python/S-MODE_analysis/data/external/aviso2.nc
2022-10-12 16:07:20.649 [ INFO] -  81.3 MB (1.0%)
2022-10-12 16:0

In [5]:
def plot_SSH_map(tind):
    plt.clf()
    ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=200))  # Orthographic
    extent = [xmin, xmax, ymin, ymax]
    day_str = np.datetime_as_string(ds.time[tind], unit='D')
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.set_title('Sea level anomaly (DUACS), '+ day_str,size = 10.)

    #plt.set_cmap(cmap=plt.get_cmap('nipy_spectral'))
    plt.set_cmap(cmap=plt.get_cmap('turbo'))
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    #gl.xlocator = matplotlib.ticker.MaxNLocator(10)
    #gl.xlocator = matplotlib.ticker.AutoLocator
    # gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(0, 360 ,30))

    cs = ax.contourf(ds.longitude,ds.latitude,np.squeeze(ds.sla.isel(time=tind)), levels, extend='both', transform=ccrs.PlateCarree())
    # cs = ax.pcolormesh(ds.longitude,ds.latitude,np.squeeze(ds.sla), vmin=levels[0], vmax=levels[-1], transform=ccrs.PlateCarree())
    # cb = plt.colorbar(cs,ax=ax,shrink=.8,pad=.05)
    cb = plt.colorbar(cs,fraction = 0.022,extend='both')
    cb.set_label('SLA [m]',fontsize = 10)
    ax.coastlines()
    ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')

    functions.plot_ops_area(ax,transform=ccrs.PlateCarree(),color='w')

    # Add a 10 km scale bar
    km_per_deg_lat=gsw.geostrophy.distance((121.7,121.7), (37,38))/1000
    deg_lat_equal_10km=10/km_per_deg_lat
    x0 = -122 
    y0 = 37
    ax.plot(x0+np.asarray([0, 0]),y0+np.asarray([0.,deg_lat_equal_10km[0]]),transform=ccrs.PlateCarree(),color='k',zorder=3)
    ax.text(x0+1/60, y0+.15/60, '10 km', fontsize=6,transform=ccrs.PlateCarree())

    u = np.squeeze(ds.ugos.isel(time=tind)) #dtype=object
    v = np.squeeze(ds.vgos.isel(time=tind))
    ax.quiver(ds.longitude.values,ds.latitude.values, u.values, v.values,  scale=5, transform=ccrs.PlateCarree())
    x0 = -122.5 
    y0 = 38.5
    ax.quiver(np.array([x0]), np.array([y0]), -np.array([0.25/np.sqrt(2)],), np.array([0.25/np.sqrt(2)]), scale=3, transform=ccrs.PlateCarree(),zorder=3)
    ax.text(x0+3/60, y0+.15/60, '0.25 m/s', fontsize=6,transform=ccrs.PlateCarree())

    if savefig:
        plt.savefig(__figdir__+'SLA'+str(tind)+'.'+plotfiletype,**savefig_args)

In [6]:
# url = 'https://nrt.cmems-du.eu/thredds/dodsC/dataset-duacs-nrt-global-merged-allsat-phy-l4'
file = '../data/external/aviso2.nc'
ds = xr.open_dataset(file)

In [7]:
ds

<xarray.Dataset>
Dimensions:    (time: 424, latitude: 37, longitude: 81)
Coordinates:
  * latitude   (latitude) float32 33.88 34.12 34.38 34.62 ... 42.38 42.62 42.88
  * time       (time) datetime64[ns] 2021-08-15 2021-08-16 ... 2022-10-12
  * longitude  (longitude) float32 -140.1 -139.9 -139.6 ... -120.6 -120.4 -120.1
Data variables:
    adt        (time, latitude, longitude) float64 ...
    ugos       (time, latitude, longitude) float64 ...
    vgos       (time, latitude, longitude) float64 ...
    vgosa      (time, latitude, longitude) float64 ...
    crs        int32 ...
    err_vgosa  (time, latitude, longitude) float64 ...
    err_ugosa  (time, latitude, longitude) float64 ...
    sla        (time, latitude, longitude) float64 ...
    ugosa      (time, latitude, longitude) float64 ...
Attributes: (12/45)
    Conventions:                                    CF-1.6
    FROM_ORIGINAL_FILE__Metadata_Conventions:       Unidata Dataset Discovery...
    cdm_data_type:                                  Grid
    comment:                                        Sea Surface Height measur...
    contact:                                        servicedesk.cmems@mercato...
    creator_email:                                  servicedesk.cmems@mercato...
    ...                                             ...
    time_coverage_duration:                         P1D
    time_coverage_end:                              2022-10-12T12:00:00Z
    time_coverage_resolution:                       P1D
    time_coverage_start:                            2022-10-11T12:00:00Z
    title:                                          NRT merged all satellites...
    _CoordSysBuilder:                               ucar.nc2.dataset.conv.CF1...

In [8]:
fig = plt.figure()
tind=-1
plot_SSH_map(tind)

{'transform': <cartopy.crs.PlateCarree object at 0x7f6d79cf0ef0>, 'color': 'w'}


In [9]:
fig = plt.figure()
for tind in range(len(ds.time)):
    plot_SSH_map(tind)

{'transform': <cartopy.crs.PlateCarree object at 0x7f6d7879b220>, 'color': 'w'}
{'transform': <cartopy.crs.PlateCarree object at 0x7f6d79aa09f0>, 'color': 'w'}


/home/jtomf/miniconda3/lib/python3.9/site-packages/cartopy/mpl/gridliner.py:741: UserWarning: The labels of this gridliner do not belong to the gridliner axes
  warnings.warn('The labels of this gridliner do not belong to '


{'transform': <cartopy.crs.PlateCarree object at 0x7f6d798cbef0>, 'color': 'w'}


/home/jtomf/miniconda3/lib/python3.9/site-packages/cartopy/mpl/gridliner.py:741: UserWarning: The labels of this gridliner do not belong to the gridliner axes
  warnings.warn('The labels of this gridliner do not belong to '


{'transform': <cartopy.crs.PlateCarree object at 0x7f6d7952fc70>, 'color': 'w'}
{'transform': <cartopy.crs.PlateCarree object at 0x7f6d79952b30>, 'color': 'w'}


...

/home/jtomf/miniconda3/lib/python3.9/site-packages/cartopy/mpl/gridliner.py:741: UserWarning: The labels of this gridliner do not belong to the gridliner axes
  warnings.warn('The labels of this gridliner do not belong to '


{'transform': <cartopy.crs.PlateCarree object at 0x7f6d787954f0>, 'color': 'w'}
{'transform': <cartopy.crs.PlateCarree object at 0x7f6d79c38040>, 'color': 'w'}


/home/jtomf/miniconda3/lib/python3.9/site-packages/cartopy/mpl/gridliner.py:741: UserWarning: The labels of this gridliner do not belong to the gridliner axes
  warnings.warn('The labels of this gridliner do not belong to '


{'transform': <cartopy.crs.PlateCarree object at 0x7f6d79a560e0>, 'color': 'w'}


/home/jtomf/miniconda3/lib/python3.9/site-packages/cartopy/mpl/gridliner.py:741: UserWarning: The labels of this gridliner do not belong to the gridliner axes
  warnings.warn('The labels of this gridliner do not belong to '


In [10]:
__figdir__+'SLA'+str(tind)+'.'+plotfiletype

'../plots/SSH_plots/SLA423.png'

In [11]:
pwd

'/home/jtomf/Python/S-MODE_analysis/code_IOP1'

In [12]:
# !ffmpeg -i SLA%d.png -r 10 video5.avi

## Extract a time series at a point or at a series of points

Try making input a list of lat, lon, times to interpolate to.  I could make that a function and then iterate over times.

So, I want to interpolate DUACS u,v to a given time, lon, lat.

Will use xarray interp package, following:  
https://docs.xarray.dev/en/stable/user-guide/interpolation.htmlhttps://docs.xarray.dev/en/stable/user-guide/interpolation.html


In [13]:
new_lon = -126.1
new_lat = 37.1
new_time = ds.time[-3]
dsi = ds.interp(time=new_time,latitude=new_lat, longitude=new_lon)

/home/jtomf/miniconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/jtomf/miniconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/jtomf/miniconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/jtomf/miniconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated an

In [14]:
dsi

<xarray.Dataset>
Dimensions:    ()
Coordinates:
    time       datetime64[ns] 2022-10-10
    latitude   float64 37.1
    longitude  float64 -126.1
Data variables:
    adt        float64 0.5523
    ugos       float64 -0.04718
    vgos       float64 0.05769
    vgosa      float64 0.1003
    crs        int32 -2147483647
    err_vgosa  float64 0.0778
    err_ugosa  float64 0.05248
    sla        float64 0.04332
    ugosa      float64 -0.05853
Attributes: (12/45)
    Conventions:                                    CF-1.6
    FROM_ORIGINAL_FILE__Metadata_Conventions:       Unidata Dataset Discovery...
    cdm_data_type:                                  Grid
    comment:                                        Sea Surface Height measur...
    contact:                                        servicedesk.cmems@mercato...
    creator_email:                                  servicedesk.cmems@mercato...
    ...                                             ...
    time_coverage_duration:                         P1D
    time_coverage_end:                              2022-10-12T12:00:00Z
    time_coverage_resolution:                       P1D
    time_coverage_start:                            2022-10-11T12:00:00Z
    title:                                          NRT merged all satellites...
    _CoordSysBuilder:                               ucar.nc2.dataset.conv.CF1...